In [2]:
import openai
import pandas as pd
from openai import OpenAI
import random
from tqdm import tqdm
import numpy as np

In [12]:
# Function to generate a resume
def generate_resume(persona, job, client):

    p_str = "\n"
    for k, v in persona.to_dict().items():
        if k == "person_id":
            continue
        p_str += "      {}: {}\n".format(k, v)

    prompt = f"""
    Generate a realistic 1/2 page resume for the following person for the job of {job}, in the New York metro area. 
    Make sure they are qualified.
    As much as possible, include names of real companies and universities. 
    Do not include a name or email, instead give placeholders [NAME] and [EMAIL].  
    DO NOT INCLUDE ANY OTHER PLACEHOLDERS, PLEASE FILL IN THE REST OF THE INFORMATION.
    Only return the resume, do not give any explanation or further words.
    \n
    Person:{p_str}
    """
    
    completion = client.chat.completions.create(
        # model='gpt-4o-mini',
        model='gpt-4o-2024-08-06',
        temperature=1.2,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    
    resume = completion.choices[0].message.content
    return resume

In [ ]:
# Initialize the OpenAI API
api_key = ''

client = OpenAI(
  api_key=api_key,  # this is also the default, it can be omitted
)

# Load generated personas
n_resumes_to_generate = 250
personas_df = pd.read_csv("./data/generated_personas.csv")[:n_resumes_to_generate]
print("No. personas:", len(personas_df))
personas_df.head()

No. personas: 250


,Age,Sex,Education,Class of Worker,Marital Status,Place of Birth,Big Five Scores 1,Big Five Scores 2,Defining Quirks,Personal Time,...,Fertility,Income Bracket,Housing Situation,Relationship with Technology,Hobbies,Communication Style,Risk Tolerance,Travel Frequency,Pet Ownership,person_id
0,36,Male,Bachelor's Degree,Public,Single,Connecticut,High openness,High agreeableness,Introverted,Gaming,...,Has children,High income,Rents,Tech-savvy,Photography,Open,Risk-averse,Rare traveler,Owns a dog,0
1,27,Female,Associate's Degree,Self-Employed,Divorced,New Jersey,High neuroticism,High openness,Very social,Cooking,...,Has children,Upper-middle income,Owns home,Tech-averse,Gardening,Direct,Moderate risk-taker,Rare traveler,Owns a cat,1
2,43,Female,Master's Degree,Self-Employed,Divorced,Canada,High conscientiousness,High agreeableness,Loves puzzles,Gaming,...,Undecided,Low income,Owns home,Familiar,Crafting,Humorous,Moderate risk-taker,Frequent traveler,Owns other pets,2
3,30,Female,Bachelor's Degree,Self-Employed,Divorced,Florida,High openness,High openness,Extremely organized,Reading,...,Undecided,Low income,Rents,Tech-averse,Photography,Direct,Moderate risk-taker,Rare traveler,Owns other pets,3
4,32,Female,Associate's Degree,Public,Divorced,Canada,High conscientiousness,High conscientiousness,Always punctual,Cooking,...,Planning to have children,Low income,Owns home,Tech-averse,Hiking,Open,High risk-taker,Occasional traveler,Owns a cat,4


In [14]:
# Generate resumes
resumes = []
resumes_df = None

jobs = [
    "Police Officer",  
    "Social Worker",
]

for p_i, persona in tqdm(personas_df.iterrows()):
    for job in jobs:
        resume = generate_resume(persona, job, client)
        resumes.append({**persona, "job": job, "resume": resume})

        # Convert to DataFrame and save to CSV
        resumes_df = pd.DataFrame(resumes)
        resumes_df.to_csv("./data/generated_resumes_with_personas_no_race.csv", index=False)



250it [49:20, 11.84s/it]
